# Real-time Implementation and Automation of Trading Strategies

_Disclaimer: <br>
The following illustrative examples are for general information and educational purposes only. <br>
It is neither investment advice nor a recommendation to trade, invest or take whatsoever actions.<br>
The below code should only be used in combination with the Binance Spot Testnet and NOT with a Live Trading Account._

__Insert your Spot Testnet Credentials here__:

In [1]:
api_key="UW1N8cz2r5ZIxFsBiF4WXDYyNJIbMtqHema1USdfSvUbQYG4wU2Ad8ZZn9Y2xfPa"
secret_key="iWpfQkCcr9OYrsZD8iddiTj6AYDk3tg6Paw1S0dXTlg4T0wapzb53bLR4fWxItvU"

## Getting Started

In [2]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd

In [3]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True) # Testnet!!!

In [4]:
client

In [5]:
client.get_account() # Test account details

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1674368460084,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.95393500', 'locked': '0.30000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '4219.79156372', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

## Recap: Streaming and Collecting Real-Time Candles

In [6]:
twm = ThreadedWebsocketManager()
twm.start()

__Kline/Candlestick Stream__

In [7]:
df = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
df

,Open,High,Low,Close,Volume,Complete


In [8]:
def stream_candles(msg):
    ''' define how to process incoming WebSocket messages '''
    
    # extract the required items from msg
    event_time = pd.to_datetime(msg["E"], unit = "ms")
    start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
    first   = float(msg["k"]["o"])
    high    = float(msg["k"]["h"])
    low     = float(msg["k"]["l"])
    close   = float(msg["k"]["c"])
    volume  = float(msg["k"]["v"])
    complete=       msg["k"]["x"]
    
    # print out
    print("Time: {} | Price: {}".format(event_time, close))
    
    # feed df (add new bar / update latest bar)
    df.loc[start_time] = [first, high, low, close, volume, complete]

valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

In [9]:
twm.start_kline_socket(callback = stream_candles, symbol = "BTCUSDT", interval = "1m")

'btcusdt@kline_1m'

Time: 2023-01-22 06:21:42.077000 | Price: 22888.3
Time: 2023-01-22 06:21:44.120000 | Price: 22887.67
Time: 2023-01-22 06:21:46.123000 | Price: 22887.92
Time: 2023-01-22 06:21:48.135000 | Price: 22885.02
Time: 2023-01-22 06:21:50.367000 | Price: 22883.14
Time: 2023-01-22 06:21:52.693000 | Price: 22883.57
Time: 2023-01-22 06:21:54.907000 | Price: 22883.61
Time: 2023-01-22 06:21:57.007000 | Price: 22884.35
Time: 2023-01-22 06:21:59.525000 | Price: 22884.04
Time: 2023-01-22 06:22:00.003000 | Price: 22884.08
Time: 2023-01-22 06:22:02.010000 | Price: 22884.42
Time: 2023-01-22 06:22:04.178000 | Price: 22887.07
Time: 2023-01-22 06:22:06.359000 | Price: 22887.44


In [10]:
twm.stop()

Time: 2023-01-22 06:22:08.376000 | Price: 22887.62


In [11]:
df

,Open,High,Low,Close,Volume,Complete
2023-01-22 06:21:00,22886.95,22889.99,22882.47,22884.08,98.9994,True
2023-01-22 06:22:00,22884.08,22888.00,22883.75,22887.62,11.2828,False


Time: 2023-01-22 06:22:10.380000 | Price: 22888.66


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Open      0 non-null      object
 1   High      0 non-null      object
 2   Low       0 non-null      object
 3   Close     0 non-null      object
 4   Volume    0 non-null      object
 5   Complete  0 non-null      object
dtypes: object(6)
memory usage: 0.0+ bytes


## A LongOnlyTrader Class

In [12]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [13]:
trader = LongOnlyTrader(symbol = "BTCUSDT", bar_length = "1m")
trader

In [14]:
trader.symbol

'BTCUSDT'

In [15]:
trader.data

,Open,High,Low,Close,Volume,Complete


In [19]:
trader.available_intervals

['1m',
 '3m',
 '5m',
 '15m',
 '30m',
 '1h',
 '2h',
 '4h',
 '6h',
 '8h',
 '12h',
 '1d',
 '3d',
 '1w',
 '1M']

In [18]:
trader.start_trading()

Time: 2023-01-22 06:23:35.040000 | Price: 22916.71


Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() done, defined at c:\Users\minbh\miniconda3\envs\myenv3\lib\site-packages\binance\streams.py:122> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x0000026C80DAEEB8>()]>>


Time: 2023-01-22 06:23:37.124000 | Price: 22914.45
Time: 2023-01-22 06:23:39.186000 | Price: 22915.83
Time: 2023-01-22 06:23:41.253000 | Price: 22915.82
Time: 2023-01-22 06:23:43.267000 | Price: 22914.44
Time: 2023-01-22 06:23:45.395000 | Price: 22915.82
Time: 2023-01-22 06:23:47.437000 | Price: 22916.72
Time: 2023-01-22 06:23:49.576000 | Price: 22914.53
Time: 2023-01-22 06:23:51.682000 | Price: 22916.59


In [19]:
trader.twm.stop()

Time: 2023-01-22 06:23:53.838000 | Price: 22915.37


In [20]:
trader.data

,Open,High,Low,Close,Volume,Complete
2023-01-22 06:23:00,22907.67,22921.0,22901.07,22915.37,205.65524,False


## Working with historical data and real-time data (Part 1)

For many Trading Strategies, we need __historical Data__ (e.g. last 2 hours) to create Strategy Features. <br>

- Option 1: we could __wait__ until our __stream collects sufficient Data__ -> __not practicable__
- Option 2: we __pull historical data__ first and __merge__ with live data -> __best practice__

In [21]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta

In [22]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [23]:
now = datetime.utcnow()
now

datetime.datetime(2023, 1, 22, 6, 24, 49, 765531)

__Example: Last two days until now__

In [24]:
past = now - timedelta(days = 2)
past

datetime.datetime(2023, 1, 20, 6, 24, 49, 765531)

In [25]:
str(past)

'2023-01-20 06:24:49.765531'

In [26]:
def get_most_recent(symbol, interval, days):
    
    now = datetime.utcnow()
    past = str(now - timedelta(days = days))
    
    bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                        start_str = past, end_str = None, limit = 1000)
    df = pd.DataFrame(bars)
    df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
    df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                  "Clos Time", "Quote Asset Volume", "Number of Trades",
                  "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
    df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
    df.set_index("Date", inplace = True)
    for column in df.columns:
        df[column] = pd.to_numeric(df[column], errors = "coerce")
    df["Complete"] = [True for row in range(len(df)-1)] + [False]
    
    return df

In [29]:
df = get_most_recent(symbol = "BTCUSDT", interval = "1m", days = 2)
df

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-01-20 06:20:00,20945.53,20949.10,20941.24,20943.19,5.459858,True
2023-01-20 06:21:00,20942.59,20942.92,20933.09,20936.94,4.480482,True
2023-01-20 06:22:00,20936.19,20965.99,20933.30,20964.86,7.639699,True
2023-01-20 06:23:00,20964.14,20969.05,20957.36,20969.05,5.893570,True
2023-01-20 06:24:00,20968.78,20975.47,20967.66,20974.84,5.883697,True
...,...,...,...,...,...,...
2023-01-22 06:15:00,22880.06,22880.06,22867.14,22870.98,4.710699,True
2023-01-22 06:16:00,22870.48,22873.06,22866.10,22869.51,4.212542,True
2023-01-22 06:17:00,22869.84,22870.68,22864.22,22864.22,3.813673,True


valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M

In [27]:
get_most_recent(symbol = "BTCUSDT", interval = "1m", days = 2/24)

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-01-22 04:26:00,22772.88,22775.84,22766.15,22775.84,5.497911,True
2023-01-22 04:27:00,22776.17,22782.51,22773.90,22779.82,4.619741,True
2023-01-22 04:28:00,22779.89,22787.67,22779.89,22784.47,4.770564,True
2023-01-22 04:29:00,22785.41,22791.21,22778.80,22791.12,4.786807,True
2023-01-22 04:30:00,22791.02,22791.02,22772.32,22773.89,4.955769,True
...,...,...,...,...,...,...
2023-01-22 06:21:00,22886.30,22888.70,22883.57,22884.57,4.635869,True
2023-01-22 06:22:00,22884.57,22905.06,22884.57,22905.06,6.834813,True
2023-01-22 06:23:00,22905.49,22918.81,22891.21,22908.48,7.393078,True


## Working with historical data and real-time data (Part 2)

In [30]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length):
        self.symbol = symbol
        self.bar_length = bar_length
        # self.data = pd.DataFrame(columns = ["Open", "High", "Low", "Close", "Volume", "Complete"])
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days) # NEW!
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days): # NEW!
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df # Create self.data
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print("Time: {} | Price: {}".format(event_time, close))
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]

In [31]:
trader = LongOnlyTrader(symbol = "BTCUSDT", bar_length = "1m")
trader

In [32]:
trader.start_trading(historical_days = 2)

Time: 2023-01-22 06:25:53.799000 | Price: 22927.44
Time: 2023-01-22 06:25:55.802000 | Price: 22930.83
Time: 2023-01-22 06:25:57.945000 | Price: 22931.3
Time: 2023-01-22 06:25:59.954000 | Price: 22931.73
Time: 2023-01-22 06:26:00.003000 | Price: 22932.14
Time: 2023-01-22 06:26:02.026000 | Price: 22928.67
Time: 2023-01-22 06:26:04.043000 | Price: 22930.11
Time: 2023-01-22 06:26:06.106000 | Price: 22929.13
Time: 2023-01-22 06:26:08.110000 | Price: 22929.53
Time: 2023-01-22 06:26:10.118000 | Price: 22925.42
Time: 2023-01-22 06:26:12.183000 | Price: 22922.49
Time: 2023-01-22 06:26:14.253000 | Price: 22922.33


In [34]:
trader.twm.stop()

In [35]:
trader.data.tail(15)

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-01-22 06:12:00,22876.26,22884.21,22876.26,22884.21,4.247603,True
2023-01-22 06:13:00,22884.21,22885.30,22883.71,22884.08,2.372171,True
2023-01-22 06:14:00,22883.57,22883.57,22871.69,22880.06,5.801026,True
2023-01-22 06:15:00,22880.06,22880.06,22867.14,22870.98,4.710699,True
2023-01-22 06:16:00,22870.48,22873.06,22866.10,22869.51,4.212542,True
2023-01-22 06:17:00,22869.84,22870.68,22864.22,22864.22,3.813673,True
2023-01-22 06:18:00,22864.65,22872.78,22864.65,22869.61,3.915043,True
2023-01-22 06:19:00,22869.68,22884.97,22869.68,22884.97,5.231151,True
2023-01-22 06:20:00,22885.29,22886.92,22880.25,22886.32,4.687057,True


## Adding a Long-Only Strategy

Let´s implement a Long-only Price & Volume Strategy with the following Parameters:

- bar_length = __1m__
- Return Threshold: __0%__
- Low and High Volume Change Threshold: __[-3.0, 3.0]__
- effectively: very simple Contrarian Strategy, sell (buy) if most recent 1m return was positive (negative)
- not profitable but many position changes & trades (good for demonstration purposes)

In [36]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [48]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [49]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
        
    def define_strategy(self): # "strategy-specific"
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
        

In [50]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]

In [51]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length,
                        return_thresh = return_thresh, volume_thresh = volume_thresh)

In [52]:
trader.start_trading(historical_days = 1/24)

.....................................

In [53]:
trader.twm.stop()

.

In [54]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2023-01-22 05:30:00,22858.14,22869.80,22858.13,22869.80,5.482772,True
2023-01-22 05:31:00,22869.70,22872.61,22868.52,22868.52,3.996905,True
2023-01-22 05:32:00,22868.29,22869.49,22864.70,22865.90,5.008088,True
2023-01-22 05:33:00,22866.01,22872.70,22864.99,22864.99,5.026085,True
2023-01-22 05:34:00,22865.19,22865.63,22854.41,22854.41,5.413332,True
...,...,...,...,...,...,...
2023-01-22 06:26:00,22931.64,22931.64,22918.15,22919.06,4.461063,True
2023-01-22 06:27:00,22918.22,22919.26,22909.65,22915.82,5.072951,True
2023-01-22 06:28:00,22915.82,22916.96,22909.80,22915.50,4.231326,True


In [55]:
trader.prepared_data

,Close,Volume,returns,vol_ch,position
Date,,,,,
2023-01-22 05:30:00,22869.80,5.482772,NaN,NaN,1
2023-01-22 05:31:00,22868.52,3.996905,-0.000056,-0.316091,1
2023-01-22 05:32:00,22865.90,5.008088,-0.000115,0.225534,1
2023-01-22 05:33:00,22864.99,5.026085,-0.000040,0.003587,1
2023-01-22 05:34:00,22854.41,5.413332,-0.000463,0.074223,1
2023-01-22 05:35:00,22864.82,6.188190,0.000455,0.133778,0
2023-01-22 05:36:00,22864.36,4.850093,-0.000020,-0.243645,1
2023-01-22 05:37:00,22860.11,4.496665,-0.000186,-0.075662,1
2023-01-22 05:38:00,22864.02,5.621752,0.000171,0.223307,0


## Placing Orders and Executing Trades

In [56]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [57]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [58]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units # NEW
        self.position = position # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades() # NEW!!!
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): # NEW!
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                print("GOING LONG")
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                print("GOING NEUTRAL")
            self.position = 0

In [59]:
client.get_account() 

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1674368460084,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.95393500', 'locked': '0.30000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '4219.79156372', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [60]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [61]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = position)

In [ ]:
trader.start_trading(historical_days = 1/24)

........................

Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() running at c:\Users\minbh\miniconda3\envs\myenv3\lib\site-packages\binance\streams.py:130> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x0000026C80E5F558>()]>>
Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() running at c:\Users\minbh\miniconda3\envs\myenv3\lib\site-packages\binance\streams.py:130> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x0000026C80F80B88>()]>>
Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() running at c:\Users\minbh\miniconda3\envs\myenv3\lib\site-packages\binance\streams.py:130> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x0000026C80E5F1F8>()]>>


...GOING LONG
.....................

In [68]:
trader.twm.stop()

In [69]:
trader.prepared_data.tail(20)

,Close,Volume,returns,vol_ch,position
Date,,,,,
2023-01-22 06:13:00,22884.08,2.372171,-0.000006,-0.582549,1
2023-01-22 06:14:00,22880.06,5.801026,-0.000176,0.894229,1
2023-01-22 06:15:00,22870.98,4.710699,-0.000397,-0.208198,1
2023-01-22 06:16:00,22869.51,4.212542,-0.000064,-0.111770,1
2023-01-22 06:17:00,22864.22,3.813673,-0.000231,-0.099473,1
2023-01-22 06:18:00,22869.61,3.915043,0.000236,0.026234,0
2023-01-22 06:19:00,22884.97,5.231151,0.000671,0.289805,0
2023-01-22 06:20:00,22886.32,4.687057,0.000059,-0.109826,0
2023-01-22 06:21:00,22884.57,4.635869,-0.000076,-0.010981,1


.

In [70]:
client.get_account() 

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1674369180092,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.96393500', 'locked': '0.30000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3990.37436372', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

## Trade Monitoring and Reporting

In [71]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [72]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [73]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 # NEW
        self.trade_values = [] # NEW
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):  # NEW
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

In [74]:
client.get_account()

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1674369180092,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.96393500', 'locked': '0.30000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3990.37436372', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

In [75]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [76]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = position)

In [ ]:
trader.start_trading(historical_days = 1/24)

Task was destroyed but it is pending!
task: <Task pending coro=<ReconnectingWebsocket._read_loop() done, defined at c:\Users\minbh\miniconda3\envs\myenv3\lib\site-packages\binance\streams.py:122> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x0000026C8128FC18>()]>>


.............

----------------------------------------------------------------------------------------------------
2023-01-22 06:35:00.078000 | GOING LONG
2023-01-22 06:35:00.078000 | Base_Units = 0.01 | Quote_Units = 229.4102 | Price = 22941.02 
2023-01-22 06:35:00.078000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

............

In [81]:
trader.twm.stop()

In [82]:
trader.prepared_data.tail(10)

,Close,Volume,returns,vol_ch,position
Date,,,,,
2023-01-22 06:25:00,22932.14,7.148523,0.000355,0.008233,0
2023-01-22 06:26:00,22919.06,4.461063,-0.000571,-0.471519,1
2023-01-22 06:27:00,22915.82,5.072951,-0.000141,0.128536,1
2023-01-22 06:28:00,22915.50,4.231326,-0.000014,-0.181407,1
2023-01-22 06:29:00,22920.15,5.634739,0.000203,0.286435,0
2023-01-22 06:30:00,22919.04,5.168942,-0.000048,-0.086283,1
2023-01-22 06:31:00,22937.76,6.504303,0.000816,0.229796,0
2023-01-22 06:32:00,22940.96,5.629017,0.000139,-0.144529,0
2023-01-22 06:33:00,22943.89,5.349369,0.000128,-0.050956,0


.

In [83]:
trader.trade_values

[-229.4102]

In [84]:
np.sum(trader.trade_values)

-229.4102

In [85]:
client.get_account() 

{'makerCommission': 0,
 'takerCommission': 0,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'commissionRates': {'maker': '0.00000000',
  'taker': '0.00000000',
  'buyer': '0.00000000',
  'seller': '0.00000000'},
 'canTrade': True,
 'canWithdraw': False,
 'canDeposit': False,
 'brokered': False,
 'requireSelfTradePrevention': False,
 'updateTime': 1674369300078,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BNB',
   'free': '1000.00000000',
   'locked': '0.00000000'},
  {'asset': 'BTC', 'free': '0.97393500', 'locked': '0.30000000'},
  {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
  {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '3760.96416372', 'locked': '0.00000000'},
  {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}],
 'permissions': ['SPOT']}

## More ways to (automatically) stop a trading Session

In [102]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [103]:
client = Client(api_key = api_key, api_secret = secret_key, tld = "com", testnet = True)

In [113]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        self.units = units
        self.position = position
        self.trades = 0 
        self.trade_values = [] 
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
    
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # stop trading session
        if event_time >= datetime(2023, 1, 22, 6, 50):
            self.twm.stop()
            if self.position != 0:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL AND STOP") 
                self.position = 0
            else: 
                print("STOP")
        
        #more stop examples:
        # if self.trades >= xyz
        # if self.cum_profits <> xyz
        
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
        
    def define_strategy(self):
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
    
    def execute_trades(self): 
        if self.prepared_data["position"].iloc[-1] == 1: # if position is long -> go/stay long
            if self.position == 0:
                order = client.create_order(symbol = self.symbol, side = "BUY", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING LONG")  
            self.position = 1
        elif self.prepared_data["position"].iloc[-1] == 0: # if position is neutral -> go/stay neutral
            if self.position == 1:
                order = client.create_order(symbol = self.symbol, side = "SELL", type = "MARKET", quantity = self.units)
                self.report_trade(order, "GOING NEUTRAL")  
            self.position = 0
    
    def report_trade(self, order, going):
        
        # extract data from order object
        side = order["side"]
        time = pd.to_datetime(order["transactTime"], unit = "ms")
        base_units = float(order["executedQty"])
        quote_units = float(order["cummulativeQuoteQty"])
        price = round(quote_units / base_units, 5)
        
        # calculate trading profits
        self.trades += 1
        if side == "BUY":
            self.trade_values.append(-quote_units)
        elif side == "SELL":
            self.trade_values.append(quote_units) 
        
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) 
            self.cum_profits = round(np.sum(self.trade_values), 3)
        else: 
            real_profit = 0
            self.cum_profits = round(np.sum(self.trade_values[:-1]), 3)
        
        # print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going)) 
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {} ".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, self.cum_profits))
        print(100 * "-" + "\n")

In [114]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]
units = 0.01
position = 0

In [115]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length, return_thresh = return_thresh,
                        volume_thresh = volume_thresh, units = units, position = position)

In [ ]:
trader.start_trading(historical_days = 1/24)

...........

----------------------------------------------------------------------------------------------------
2023-01-22 06:43:00.079000 | GOING LONG
2023-01-22 06:43:00.079000 | Base_Units = 0.01 | Quote_Units = 229.6466 | Price = 22964.66 
2023-01-22 06:43:00.079000 | Profit = 0 | CumProfits = 0.0 
----------------------------------------------------------------------------------------------------

..................................................................................................................................

----------------------------------------------------------------------------------------------------
2023-01-22 06:48:00.186000 | GOING NEUTRAL
2023-01-22 06:48:00.186000 | Base_Units = 0.01 | Quote_Units = 229.5198 | Price = 22951.98 
2023-01-22 06:48:00.186000 | Profit = -0.127 | CumProfits = -0.127 
----------------------------------------------------------------------------------------------------

......................................................

In [118]:
trader.cum_profits

-0.127

In [119]:
np.sum(trader.trade_values)

-229.6052